In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import re
import sys
import spacy
import pickle
import numpy as np

from glob import glob
# from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
from tqdm import tqdm




import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset 

In [2]:
# spacy.load("en_core_web_sm")

In [3]:
from utils import *
from data_utils import Vocabulary, tokenizer
from train_utils import train
# import torch
# from torch.autograd import Variable

# # from glob import glob
# from tqdm import tqdm
# from torch.utils.data import Dataset



# def to_var(x, volatile=False):
#     x = x.cuda() if torch.cuda.is_available() else x
#     return Variable(x, volatile=volatile)


# def detach(x):
#     """ Detach hidden states from their history."""
#     return Variable(x.data) if type(x) == Variable else tuple(detach(v) for v in x)


# # NLP = spacy.load('en')
# NLP = spacy.load("en_core_web_sm")


# def tokenizer(text):
#     text = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’;]", " ", str(text))
#     text = re.sub(r"[ ]+", " ", text)
#     text = re.sub(r"\!+", "!", text)
#     text = re.sub(r"\,+", ",", text)
#     text = re.sub(r"\?+", "?", text)
#     return [x.text for x in NLP.tokenizer(text) if x.text != " "]


# class Vocabulary(object):
    
#     def __init__(self, tokenizer):
#         self.tokenizer = tokenizer
#         self.word2index = {}
#         self.word2count = {}
#         self.index2word = {}
#         self.count = 0
    
#     def add_word(self, word):
#         if not word in self.word2index:
#             self.word2index[word] = self.count
#             self.word2count[word] = 1
#             self.index2word[self.count] = word
#             self.count += 1
#         else:
#             self.word2count[word] += 1
    
#     def add_sentence(self, sentence):
#         for word in self.tokenizer(sentence):
#             self.add_word(word)
            
#     def __len__(self):
#         return self.count

    
# if __name__ == '__main__':
#     pass


# def train_step(model, train_dl, criterion, optimizer, scheduler):
#     model.train()
#     scheduler.step()
    
#     # init hidden states
#     model.hidden = model.init_hidden()

#     total_acc = 0.0
#     total_loss = 0.0
#     total = 0.0
    
#     for i, (train_inputs, train_labels) in tqdm(enumerate(train_dl), 
#                                                          desc='Training', 
#                                                          total=len(train_dl)):

#         train_inputs, train_labels = to_var(train_inputs), to_var(train_labels)
#         if len(train_labels) < train_dl.batch_size: continue
        
#         # Starting each batch, we detach the hidden state from how it was previously produced.
#         # If we didn't, the model would try backpropagating all the way to start of the dataset.
#         model.hidden = detach(model.hidden)
#         model.zero_grad()
#         output = model(train_inputs.t())
        
#         loss = criterion(output, train_labels)
#         loss.backward()
#         torch.nn.utils.clip_grad_norm(model.parameters(), 0.3)        
#         optimizer.step()

#         # calculate training acc and loss
#         _, predicted = torch.max(output.data, 1)
#         total_acc += (predicted == train_labels.data).sum()
#         total_loss += loss.data[0]
#         total += len(train_labels)
        
#     return total_loss / total, total_acc / total


# def validate_step(model, valid_dl, criterion):
#     model.eval()
    
#     # init hidden states
#     model.hidden = model.init_hidden()
    
#     total_acc = 0.0
#     total_loss = 0.0
#     total = 0.0
    
#     for i, (test_inputs, test_labels) in tqdm(enumerate(valid_dl), 
#                                                        desc='Validation', 
#                                                        total=len(valid_dl)):

#         test_inputs, test_labels = to_var(test_inputs, True), to_var(test_labels, True)
#         if len(test_labels) < valid_dl.batch_size: continue

#         output = model(test_inputs.t())
#         loss = criterion(output, test_labels)

#         # calculate testing acc and loss
#         _, predicted = torch.max(output.data, 1)
#         total_acc += (predicted == test_labels.data).sum()
#         total_loss += loss.data[0]
#         total += len(test_labels)
        
#         model.hidden = detach(model.hidden)
        
#     return total_loss / total, total_acc / total


# def train(model, train_dl, valid_dl, criterion, optimizer, scheduler, num_epochs):
#     max_len, min_count = train_dl.dataset.max_len, train_dl.dataset.min_count
    
#     train_hist, valid_hist = [], []
#     best_acc, best_wts = 0.0, None

    
#     for epoch in range(num_epochs):

#         ## perform one epoch of training and validation
#         trn_loss, trn_acc = train_step(model, train_dl, criterion, optimizer, scheduler)
#         val_loss, val_acc = validate_step(model, valid_dl, criterion)

#         train_hist += [(trn_loss, trn_acc)]
#         valid_hist += [(val_loss, val_acc)]

#         # save weights
#         if val_acc > best_acc:
#             best_acc = val_acc
#             best_wts = model.state_dict().copy()
#             torch.save(best_wts, 'models/lstm-{}-{}-{}-{}-{}-{}-{:.5f}.pth'.format(
#                 epoch, max_len, min_count, model.embed_size, model.hidden_size, model.num_layers, best_acc))

#         print('[Epoch: %3d/%3d] Training Loss: %.3f, Testing Loss: %.3f, Training Acc: %.3f, Testing Acc: %.3f'
#               % (epoch + 1, num_epochs, trn_loss, val_loss, trn_acc, val_acc))

#     model.load_state_dict(best_wts)
#     return train_hist, valid_hist

In [4]:

# setup
use_gpu = torch.cuda.is_available()
NLP =spacy.load("en_core_web_sm") #spacy.load('en')  # NLP toolkit

In [5]:
text = """
Bromwell High is a cartoon comedy. 
It ran at the same time as some other programs about school life, such as 'Teachers'. 
My 35 years in the teaching profession lead me to believe that Bromwell High's 
satire is much closer to reality than is 'Teachers'. 
The scramble to survive financially, the insightful students who can see 
right through their pathetic teachers' pomp, the pettiness of the whole situation, 
all remind me of the schools I knew and their students. 
When I saw the episode in which a student repeatedly tried to burn down the school, 
I immediately recalled ......... at .......... High. 
A classic line: INSPECTOR: I'm here to sack one of your teachers. 
STUDENT: Welcome to Bromwell High. 
I expect that many adults of my age think that Bromwell High is far fetched. 
What a pity that it isn't!!!
"""

In [6]:
# حذف کاراکترهای بی استفاده
text = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’;]", " ", str(text))
print(text)

 Bromwell High is a cartoon comedy.  It ran at the same time as some other programs about school life, such as 'Teachers'.  My 35 years in the teaching profession lead me to believe that Bromwell High's  satire is much closer to reality than is 'Teachers'.  The scramble to survive financially, the insightful students who can see  right through their pathetic teachers' pomp, the pettiness of the whole situation,  all remind me of the schools I knew and their students.  When I saw the episode in which a student repeatedly tried to burn down the school,  I immediately recalled ......... at .......... High.  A classic line  INSPECTOR  I'm here to sack one of your teachers.  STUDENT  Welcome to Bromwell High.  I expect that many adults of my age think that Bromwell High is far fetched.  What a pity that it isn't!!! 


In [7]:
# تبدیل فاصله ها به یکی
text = re.sub(r"[ ]+", " ", text)
print(text)

 Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as 'Teachers'. My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is 'Teachers'. The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line INSPECTOR I'm here to sack one of your teachers. STUDENT Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!!! 


In [8]:
# حذف علامتهای تعجب اضافی
text = re.sub(r"\!+", "!", text)
print(text)

 Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as 'Teachers'. My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is 'Teachers'. The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line INSPECTOR I'm here to sack one of your teachers. STUDENT Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't! 


In [9]:
tokens = [w.text for w in NLP.tokenizer(text)]
print(tokens)

[' ', 'Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', "'", 'Teachers', "'", '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', "'", 'Teachers', "'", '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'line'

In [10]:
# Tokenizer and Vocabulary

def tokenizer(text):
    text = re.sub(r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’;]", " ", str(text))
    text = re.sub(r"[ ]+", " ", text)
    text = re.sub(r"\!+", "!", text)
    text = re.sub(r"\,+", ",", text)
    text = re.sub(r"\?+", "?", text)
    return [x.text for x in NLP.tokenizer(text) if x.text != " "]

### Install SpaCy 
Installation:
<pre>conda install -c conda-forge spacy</pre>

Download a language:
<pre>python -m spacy download en</pre>

Usage:
```python
import spacy
NLP = spacy.load('en')
```

In [11]:
data_dir = '/home/vahid/Documents/Personal/Machine Learning/DataSet/aclImdb_v1_rz_edit/dev'

vocab_path = 'vocab.pkl'

# parameters
max_len = 200
min_count = 10 # برای کلمات کمتر استفاده شده
batch_size = 50

In [12]:
os.listdir(data_dir)

['train', 'test']

In [13]:
os.listdir(f'{data_dir}/train')

['neg', 'pos']

In [14]:


all_filenames = glob(f'{data_dir}/*/*/*.txt')
num_words = [len(open(f).read().split(' ')) for f in tqdm(all_filenames)]

# print statistics
print('Min length =', min(num_words))
print('Max length =', max(num_words))

print('Mean = {:.2f}'.format(np.mean(num_words)))
print('Std  = {:.2f}'.format(np.std(num_words)))

print('mean + 2 * sigma = {:.2f}'.format(np.mean(num_words) + 2.0 * np.std(num_words)))

100%|██████████| 3004/3004 [00:00<00:00, 29426.77it/s]

Min length = 8
Max length = 1296
Mean = 234.61
Std  = 177.53
mean + 2 * sigma = 589.67


In [15]:
PAD = '<pad>'  # special symbol we use for padding text
UNK = '<unk>'  # special symbol we use for rare or unknown word

In [16]:
class TextClassificationDataset(Dataset):
    
    def __init__(self, path, tokenizer, 
                 split='train', 
                 vocab_path='vocab.pkl', 
                 max_len=100, min_count=10):
        
        self.path = path
        assert split in ['train', 'test']
        self.split = split
        self.vocab_path = vocab_path
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.min_count = min_count
        
        self.cache = {}
        self.vocab = None
        
        self.classes = []
        self.class_to_index = {}
        self.text_files = []
        
        split_path = f'{path}/{split}'
        for cls_idx, label in enumerate(os.listdir(split_path)):
            text_files = [(fname, cls_idx) for fname in glob(f'{split_path}/{label}/*.txt')]
            self.text_files += text_files
            self.classes += [label]
            self.class_to_index[label] = cls_idx
        
        self.num_classes = len(self.classes)
            
        # build vocabulary from training and validation texts
        self.build_vocab()
        
    def __getitem__(self, index):
        # read the tokenized text file and its label (neg=0, pos=1)
        fname, class_idx = self.text_files[index]
        
        if fname in self.cache:
            return self.cache[fname], class_idx
        
        # read text file 
        text = open(fname).read()
        
        # tokenize the text file
        tokens = self.tokenizer(text.lower())
        
        # padding and trimming
        if len(tokens) < self.max_len:
            num_pads = self.max_len - len(tokens)
            tokens = [PAD] * num_pads + tokens
        elif len(tokens) > self.max_len:
            tokens = tokens[:self.max_len]
            
        # numericalizing
        ids = torch.LongTensor(self.max_len)
        for i, word in enumerate(tokens):
            if word not in self.vocab.word2index:
                ids[i] = self.vocab.word2index[UNK]  # unknown words
            elif word != PAD and self.vocab.word2count[word] < self.min_count:
                ids[i] = self.vocab.word2index[UNK]  # rare words
            else:
                ids[i] = self.vocab.word2index[word]
                
        # save in cache for future use
        self.cache[fname] = ids
        
        return ids, class_idx
    
    def __len__(self):
        return len(self.text_files)
    
    def build_vocab(self):
        if not os.path.exists(self.vocab_path):
            vocab = Vocabulary(self.tokenizer)
            filenames = glob(f'{path}/*/*/*.txt')
            for filename in tqdm(filenames, desc='Building Vocab'):
                with open(filename, encoding='utf8') as f:
                    for line in f:
                        vocab.add_sentence(line.lower())

            # sort words by their frequencies
            words = [(0, PAD), (0, UNK)]
            words += sorted([(c, w) for w, c in vocab.word2count.items()], reverse=True)

            self.vocab = Vocabulary(self.tokenizer)
            for i, (count, word) in enumerate(words):
                self.vocab.word2index[word] = i
                self.vocab.word2count[word] = count
                self.vocab.index2word[i] = word
                self.vocab.count += 1

            pickle.dump(self.vocab, open(self.vocab_path, 'wb'))
        else:
            self.vocab = pickle.load(open(self.vocab_path, 'rb'))

In [17]:
train_ds = TextClassificationDataset(data_dir, tokenizer, 'train', vocab_path, max_len, min_count)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

valid_ds = TextClassificationDataset(data_dir, tokenizer, 'test', vocab_path, max_len, min_count)
valid_dl = DataLoader(valid_ds, batch_size=batch_size, shuffle=False)

In [18]:
len(train_ds)

2002

In [19]:
len(valid_ds)

1002

In [20]:
train_ds.classes

['neg', 'pos']

In [21]:
train_ds.class_to_index

{'neg': 0, 'pos': 1}

In [22]:
ids, label = train_ds[0]

print(train_ds.classes[label])
print(ids.numpy())

neg
[   82     3    20     1   290   310     3    45    73     9    28   109
   229    12   269     8  1346     6  1587  8386    15 19107     8    34
  5136    27   148    15    55    77     6    94   938     4  1091 18864
  4224  1091 22395    55  1404  2314    96   494    20    40  2398    27
    86     7    56    16    89   230    12  2022     5  3440    12     2
   520  3092   163     5    66    35   917    12     2   494     7    25
     3   426     3   143    34     1    43   157    11     4     8  2602
     6  2859 10929  8386     3   201     3    13   233  5017     8  3097
    22   220   178    91   724  5466   140     5   141     8    93    48
    28   115    11    11    16     6   455     7   635     7   135    69
    15    92    47   566    26   563   255  1161     5  1618     4     5
    39    15    13    48    28   391  1161     5  1618    51     2   397
  3233    46   240    42   809     5  3264    20    15     3    97    15
     9   620     2   354    15   100  1411    3

In [23]:
# convert back the sequence of integers into original text
print(' '.join([train_ds.vocab.index2word[int(i)] for i in ids]))

well , as <unk> once said , there really is n't any point in trying to pass a negative judgement that aspires to be objective on something that has had a great effect . la maman et la putain has surely passed into history as an influence on much of what 's been done in france and elsewhere in the past thirty years and no one interested in the history of film , certainly , should be <unk> from watching it . to express a purely subjective judgement , however , i feel compelled to disagree with almost every other review posted here and say to people do n't watch it it 's a waste of hours of your time that will just leave you feeling rather sick and angry . and by that i do n't mean sick and angry about the human condition or anything so general and profound as that , because that is exactly the line that most critics have adopted in their <unk> praise of the film an ordeal to watch in its ruthless dissection of our emotional cowardice and cruelty and so on and , if it really managed to pu

In [24]:
# print the original text
print(open(train_ds.text_files[0][0]).read())

Well, as Goethe once said, there really isn't any point in trying to pass a negative judgement that aspires to be objective on "something that has had a great effect". "La Maman et La Putain" has surely passed into history as an influence on much of what's been done in France and elsewhere in the past thirty years and no one interested in the history of film, certainly, should be dissuaded from watching it. To express a purely subjective judgement, however, I feel compelled to disagree with almost every other review posted here and say to people: "Don't watch it; it's a waste of hours of your time that will just leave you feeling rather sick and angry." And by that I don't mean "sick and angry" about "the human condition" or anything so general and profound as that, because that is exactly the line that most critics have adopted in their fulsome praise of the film - "an ordeal to watch in its ruthless dissection of our emotional cowardice and cruelty" and so on - and, if it really mana

In [25]:
vocab = train_ds.vocab
freqs = [(count, word) for (word, count) in vocab.word2count.items() if count >= min_count]
vocab_size = len(freqs) + 2  # for PAD and UNK tokens
print(f'Vocab size = {vocab_size}')

print('\nMost common words:')
for c, w in sorted(freqs, reverse=True)[:10]:
    print(f'{w}: {c}')

Vocab size = 29512

Most common words:
the: 666690
,: 543308
.: 469782
and: 324154
a: 321797
of: 289312
to: 267959
is: 217022
>: 202239
it: 187957


In [26]:
# LSTM Classifier
class LSTMClassifier(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, num_classes, batch_size):
        super(LSTMClassifier, self).__init__()
        
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(vocab_size, embed_size) # a lookup table
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, dropout=0.3, bidirectional=True)
        self.fc = nn.Sequential(
            nn.Linear(2*hidden_size, 100),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(100, num_classes)
        )
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        h = to_var(torch.zeros((2*self.num_layers, self.batch_size, self.hidden_size)))
        c = to_var(torch.zeros((2*self.num_layers, self.batch_size, self.hidden_size)))
        return h, c
    
    def forward(self, x):
        x = self.embedding(x)
        x, self.hidden = self.lstm(x, self.hidden)
        x = self.fc(x[-1])  # select the last output
        return x

In [27]:
vocab_size = 2 + len([w for (w, c) in train_ds.vocab.word2count.items() if c >= min_count])
print(vocab_size)

29512


In [28]:
# Model
# LSTM parameters
embed_size = 100
hidden_size = 256
num_layers = 1

# training parameters
lr = 0.001
num_epochs = 10

In [29]:
model = LSTMClassifier(embed_size=embed_size, 
                       hidden_size=hidden_size, 
                       vocab_size=vocab_size,
                       num_layers=num_layers,
                       num_classes=train_ds.num_classes, 
                       batch_size=batch_size)

# if use_gpu:
#     model = model.cuda()

/home/vahid/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [30]:
criterion = nn.CrossEntropyLoss()
if use_gpu:
    criterion = criterion.cuda()
    
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.7, 0.99))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.975)

In [31]:
hist = train(model, train_dl, valid_dl, criterion, optimizer, scheduler, num_epochs)

/home/vahid/anaconda3/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
Training:   0%|          | 0/41 [00:00<?, ?it/s]


TypeError: iteration over a 0-d tensor

In [33]:
# LSTM parameters
max_len = 400
min_count = 10
embed_size = 256
hidden_size = 1024
num_layers = 1


model = LSTMClassifier(embed_size=embed_size, 
                       hidden_size=hidden_size, 
                       vocab_size=vocab_size,
                       num_layers=num_layers,
                       num_classes=train_ds.num_classes, 
                       batch_size=batch_size)

# model.load_state_dict(torch.load('models/tmp/lstm-1-400-10-256-1024-1-0.87964.pth'))

if use_gpu:
    model = model.cuda()

In [34]:
word_vecs = model.embedding.weight.data =  

SyntaxError: invalid syntax (<ipython-input-34-3052a0f2d4dc>, line 1)